In [49]:
import os 
import sys
import h5py
path = os.path.dirname(os.path.dirname(os.path.realpath("__file__")))
sys.path.insert(0,path)
#print(sys.version_info)
import numpy as np
import matplotlib.pyplot as plt
import time

from utils.api_functions import *
from utils.opt_functions import *


%run -i "C:\Users\esslinger\Documents\Zemax\ZOS-API Projects\PythonZOSConnection\PythonZOSConnection.py"

fast_system(TheSystem) # this speeds up the program.

Found OpticStudio at:   %sc:\program files\zemax opticstudio
Connected to OpticStudio
Serial #:  L123525


Compare Merit Function change vs not change

In [6]:
#we can set up the merit function here.
MFE = TheSystem.MFE


for i in np.arange(10):
    MFE.LoadMeritFunction(path + '/data/Merits/LO_standard.mf')
    calc_merit(MFE)
    MFE.LoadMeritFunction(path + '/data/Merits/LO_standard_modified.mf')
    calc_merit(MFE)

In [7]:
#we can set up the merit function here.
MFE = TheSystem.MFE


MFE.LoadMeritFunction(path + '/data/Merits/LO_standard.mf')
for i in np.arange(20):
    calc_merit(MFE)


Simple System: About 50% slower when switching Merit function every time (2*1000 evaluations) (14.2s vs 10.6s)
High computational requirements (2000 Fields and LO system)(2*10 evaluations): 5 min 28s vs 5 min 13s



Compare 1x10 optimizations within ZOS to 10x1 optimization in ZOS -> extract more merit function to lens parameter data points

In [2]:
LDE = TheSystem.LDE
MFE = TheSystem.MFE
#MFE.LoadMeritFunction(path + '/data/Merits/Gauss_Lense_youtube.mf')
print(get_parameter(LDE))
print(calc_merit(MFE))
print('------')
local_optimisation(TheSystem, ZOSAPI, algorithm = 'DLS', cycles = 10)
print(calc_merit(MFE))
print(get_parameter(LDE))
print('------')

(array([ 1.00000000e+06, -1.00000000e+06,  1.12900000e+01,  2.84000000e+00,
        2.00000000e-01,  2.00000000e+00,  2.13662570e+01,  2.28000000e+00,
        4.46000000e+00,  2.54119830e+00,  2.29000000e+00,  2.00000000e-01,
        3.24000000e+00,  9.52148611e-01,  0.00000000e+00,  0.00000000e+00,
        8.44000000e+00,  0.00000000e+00,  0.00000000e+00,  9.09000000e+00,
        0.00000000e+00,  0.00000000e+00,  4.62000000e+00,  0.00000000e+00,
        0.00000000e+00,  5.19000000e+00,  0.00000000e+00,  0.00000000e+00,
        1.50000000e+00,  0.00000000e+00,  0.00000000e+00,  1.50000000e+00,
        0.00000000e+00,  0.00000000e+00,  5.19000000e+00,  0.00000000e+00,
        0.00000000e+00,  4.62000000e+00,  0.00000000e+00,  0.00000000e+00,
        9.09000000e+00,  0.00000000e+00,  0.00000000e+00,  8.44000000e+00,
        0.00000000e+00,  0.00000000e+00,  9.52148611e-01,  3.24000000e+00,
        2.00000000e-01,  2.29000000e+00,  2.54119830e+00,  4.46000000e+00,
        2.28000000e+00, 

In [3]:
print(calc_merit(MFE))
thickness, radius, merits = local_optimisation_n_single_cycles(TheSystem, ZOSAPI, algorithm = 'DLS', cycles=10)
print(merits)

0.024009539067582297
[0.02400954 0.02400954 0.02400954 0.02400954 0.02400954 0.02400954
 0.02400954 0.02400954 0.02400954 0.02400954]


Simple system: About 10 times longer if doing 10 single steps
High computational requirements (2000 Fields and LO system): 10 Steps at once: 11 min 58s
10 single steps: 15 min 56s

Test optimization within python using dual annealing as example

In [50]:
MFE = TheSystem.MFE
LDE = TheSystem.LDE
#MFE.LoadMeritFunction(path + '/data/Merits/Gauss_Lense_youtube.mf')
radius_var_status, thickness_var_status = get_variable_status(LDE) 
print(get_parameter(LDE))
print(calc_merit(MFE))
ret = opt_annealing(radius_var_status, thickness_var_status, LDE, MFE, steps = 1000)
print(ret.fun)
print(ret.x)
print(calc_merit(MFE))
print(get_parameter(LDE))

(array([        inf,  8.74665785,  0.5       , 14.00024419,  3.77696589,
        4.9001318 , 18.428     ,  3.77696589, 10.8339285 ,  0.5       ,
        6.85817491, 57.31453791,         inf]), array([         inf,  54.15324617, 152.52192094,  35.99569578,
                inf,  22.30253501,          inf, -25.68503303,
                inf, -36.98022073, 196.4173341 , -67.14755002,
                inf]))
0.14061524753143703
0.08261672928405754
[0.13634428 0.13051909]
0.08261672928405754
(array([        inf,  8.74665785,  0.5       , 14.00024419,  3.77696589,
       13.63442756, 13.05190868,  3.77696589, 10.8339285 ,  0.5       ,
        6.85817491, 57.31453791,         inf]), array([         inf,  54.15324617, 152.52192094,  35.99569578,
                inf,  22.30253501,          inf, -25.68503303,
                inf, -36.98022073, 196.4173341 , -67.14755002,
                inf]))
